Text Analytics I HWS 22/23

# Home Assignment 4 (29 pts)


Submit your solution via Ilias until 23.59pm on Tuesday, November 29th. Late submissions are accepted until 10:15am on the following day (start of the exercise), with 1/4 of the total possible points deducted from the score.

Submit your solutions in teams of 3-4 students. Unless explicitly agreed otherwise in advance, **submissions from teams with more or less members will NOT be graded**.
List all members of the team with their student ID and full name in the cell below, and submit only one notebook per team. Only submit a notebook, do not submit the dataset(s) you used or image files that you have created - these have to be created from your notebook. Also, do NOT compress/zip your submission!

You may use the code from the exercises and basic functionalities that are explained in the official documentation of Python packages without citing, __all other sources must be cited__. In case of plagiarism (copying solutions from other teams or from the internet) ALL team members will be expelled from the course without warning.

#### General guidelines:
* Make sure that your code is executable, any task for which the code does not directly run on our machine will be graded with 0 points.
* Use only packages that are automatically installed along with Anaconda, plus some additional packages that have been introduced in the context of this class.
* Ensure that the notebook does not rely on the current notebook or system state!
  * Use `Kernel --> Restart & Run All` to see if you are using any definitions, variables etc. that 
    are not in scope anymore.
  * Do not rename any of the datasets you use, and load it from the same directory that your ipynb-notebook is located in, i.e., your working directory.
* Make sure you clean up your code before submission, e.g., properly align your code, and delete every line of code that you do not need anymore, even if you may have experimented with it. Minimize usage of global variables. Do not reuse variable names multiple times!
* Ensure your code/notebook terminates in reasonable time.
* Feel free to use comments in the code. While we do not require them to get full marks, they may help us in case your code has minor errors.
* For questions that require a textual answer, please do not write the answer as a comment in a code cell, but in a Markdown cell below the code. Always remember to provide sufficient justification for all answers.
* You may create as many additional cells as you want, just make sure that the solutions to the individual tasks can be found near the corresponding assignment.
* If you have any general question regarding the understanding of some task, do not hesitate to post in the student forum in Ilias, so we can clear up such questions for all students in the course.

In [3]:
# credentials of all team members
team_members = [
   {
        'first_name': 'Qi',
        'last_name': 'Jiang',
        'student_id': 1820722
    },
    {
        'first_name': 'Jiatong',
        'last_name': 'Yu',
        'student_id': 1908029
    },
    {
        'first_name': 'Yongyi',
        'last_name': 'Zheng',
        'student_id': 1823267
    },
    {
        'first_name': 'Zihang',
        'last_name': 'Chen',
        'student_id': 1819574
    }
]

In [42]:
from typing import List, Union, Dict, Set, Tuple
from numpy.typing import NDArray
import numpy as np


import warnings
warnings.filterwarnings("ignore")

### Task 1: Term Frequency - Inverse Document Frequency (29 pts)

In this task we want to use the term frequency - inverse document frequency (tf-idf) weighting method to compare documents with each other and to queries. In the end, we will apply our method to a subset of wikipedia pages (more specifically: only the introduction sections) that are linked to from the English Wikipedia page of Mannheim.

In case you need to tokenize any sentences in the following tasks, please use a tokenizer from NLTK and not the ``string.split`` function.

__a)__ To test your implementation throughout this task, you are given the example from exercise 8. Start by implementing a function ``process_docs`` that takes the provided dictionary of documents and returns the following data structures. __(4 pts)__

- ``word2index``: a dictionary that maps each word that appears in any document to a unique integer identifier starting at 0 
- ``doc2index``: a dictionary that maps each document name (here given as the dictionary keys) to a unique integer identifier starting at 0
- ``index2doc``: a dictionary that maps each document identifier to the corresponding document name (reverse to ``doc2index``)
- ``doc_word_vectors``: a dictionary that maps each document name to a list of word ids that indicate which words appeared in the document in their order of appearance. Words that appear multiple times must also be included multiple times.

In [43]:
# example from exercise 8
d1 = "cold beer beach"
d2 = "ice cream beer beer"
d3 = "beach cold ice cream"
d4 = "cold beer frozen yogurt frozen beer"
d5 = "frozen ice ice beer ice cream"
d6 = "yogurt ice cream ice cream"

docs = {"d1": d1, "d2": d2, "d3": d3, "d4": d4, "d5": d5, "d6": d6}

In [44]:
from nltk.tokenize import word_tokenize

def process_docs(docs: Dict[str, str]) -> (Dict[str, int], Dict[str, int], Dict[int, str], Dict[str, List[int]]):
    """
    :params docs: dict that maps each document name to the document content
    :returns:
        - word2index: dict that maps each word to a unique id
        - doc2index: dict that maps each document name to a unique id
        - index2doc: dict that maps ids to their associated document name
        - doc_word_vectors: dict that maps each document name to a list of word ids that appear in it
    """
    word2index = dict()
    doc2index = dict()
    index2doc = dict()
    doc_word_vectors = dict()
    word_index = 0
    doc_index = 0
    
    for doc_name in docs.keys():
        
        # to generate word2index
        sentence = word_tokenize(docs[doc_name])
        for word in sentence:
            if word not in word2index.keys():
                word2index[word] = word_index
                word_index += 1
                
        # to generate doc2index and index2doc
        if doc_name not in doc2index.keys():
            doc2index[doc_name] = doc_index
            index2doc[doc_index] = doc_name
            doc_index +=1
    
    # to generate doc_word_vectors    
    for doc_name in docs.keys():
        sentence = word_tokenize(docs[doc_name])
        list_of_ids = [word2index[word] for word in sentence]
        doc_word_vectors[doc_name] = list_of_ids
       
    return word2index, doc2index, index2doc, doc_word_vectors

In [45]:
# The output for the provided example could look like this:

# word2index:
# {'cold': 0, 'beer': 1, 'beach': 2, 'ice': 3, 'cream': 4, 'frozen': 5, 'yogurt': 6}

# doc2index:
# {'d1': 0, 'd2': 1, 'd3': 2, 'd4': 3, 'd5': 4, 'd6': 5}

# index2doc
# {0: 'd1', 1: 'd2', 2: 'd3', 3: 'd4', 4: 'd5', 5: 'd6'}

# doc_word_vectors:
# {'d1': [0, 1, 2],
#  'd2': [3, 4, 1, 1],
#  'd3': [2, 0, 3, 4],
#  'd4': [0, 1, 5, 6, 5, 1],
#  'd5': [5, 3, 3, 1, 3, 4],
#  'd6': [6, 3, 4, 3, 4]}
print(process_docs(docs))
word2index, doc2index, index2doc, doc_word_vectors=process_docs(docs)

({'cold': 0, 'beer': 1, 'beach': 2, 'ice': 3, 'cream': 4, 'frozen': 5, 'yogurt': 6}, {'d1': 0, 'd2': 1, 'd3': 2, 'd4': 3, 'd5': 4, 'd6': 5}, {0: 'd1', 1: 'd2', 2: 'd3', 3: 'd4', 4: 'd5', 5: 'd6'}, {'d1': [0, 1, 2], 'd2': [3, 4, 1, 1], 'd3': [2, 0, 3, 4], 'd4': [0, 1, 5, 6, 5, 1], 'd5': [5, 3, 3, 1, 3, 4], 'd6': [6, 3, 4, 3, 4]})


__b)__ Set up a term-document matrix where each column corresponds to a document and each row corresponds to a word that was observed in any of the documents. The row/column indices should correspond to the word/document ids that are set in the input dicts ``word2index`` and ``doc2index``. Count how often each word appears in each document and fill the term document matrix. __(3 pts)__

_Example: The word "beer" with the word id_ ``1`` _appears two times in the document "d4" that has the document id_ ``3``. _Therefore the the entry at position_ ``[1, 3]`` _in the term-document matrix is_ ``2``.



In [46]:
def term_document_matrix(doc_word_v: Dict[str, List[int]], doc2index: Dict[str, int], word2index: Dict[str, int]) -> NDArray[NDArray[float]]:
    """
    :param doc_word_v: dict that maps each document to the list of word ids that appear in it
    :param doc2index: dict that maps each document name to a unique id
    :param word2index: dict that maps each word to a unique id
    :return: term-document matrix (each word is a row, each document is a column) that indicates the count of each word in each document 
    """
    term_document = np.zeros((len(word2index), len(doc2index)))
    for k,v in doc_word_v.items():
        for i in v:
            term_document[i,doc2index[k]]+=1
    return term_document

In [47]:
td_matrix = term_document_matrix(doc_word_vectors,doc2index,word2index)
print(td_matrix)

[[1. 0. 1. 1. 0. 0.]
 [1. 2. 0. 2. 1. 0.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 1. 0. 3. 2.]
 [0. 1. 1. 0. 1. 2.]
 [0. 0. 0. 2. 1. 0.]
 [0. 0. 0. 1. 0. 1.]]


__c)__ Implement the function ``to_tf_idf_matrix`` that takes a term-document matrix and returns the corresponding term frequency (tf) matrix. If the parameter ``idf`` is set to ``True``, the tf-matrix should further be transformed to a tf-idf matrix (i.e. every entry corresponds to the tf-idf value of the associated word and document). Your implementation should leave the input term-document matrix unchanged. __(3 pts)__

Recall the following formulas:

$\begin{equation}
  tf_{t,d} =
    \begin{cases}
      1+log_{10}\text{count}(t,d) & \text{if count$(t, d) > 0$}\\
      0 & \text{otherwise}
    \end{cases}       
\end{equation}$  

$idf_t = log_{10}(\frac{N}{df_i}) $  

$tf-idf_{t,d} = tf_{t,d} \cdot idf_t$

In [48]:
def to_tf_idf_matrix(td_matrix: NDArray[NDArray[float]], idf: bool=True) -> NDArray[NDArray[float]]:
    """
    :param td_matrix: term-document matrix 
    :param idf: computes the tf-idf matrix if True, otherwise computes only the tf matrix
    :return: matrix with tf(-idf) values for each word-document pair 
    """
    tf_matrix = np.where(td_matrix > 0, 1 + np.log10(td_matrix), 0)
    if idf:
        N = td_matrix.shape[1]
        idf = []
        for i in range(td_matrix.shape[0]):
            idf.append(np.log10(N/sum(np.where(td_matrix[i] > 0, 1, 0))))
            for j in range(td_matrix.shape[1]):
                tf_matrix[i,j] *= idf[i]
    return tf_matrix

In [49]:
to_tf_idf_matrix(td_matrix, False)

array([[1.        , 0.        , 1.        , 1.        , 0.        ,
        0.        ],
       [1.        , 1.30103   , 0.        , 1.30103   , 1.        ,
        0.        ],
       [1.        , 0.        , 1.        , 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 1.        , 0.        , 1.47712125,
        1.30103   ],
       [0.        , 1.        , 1.        , 0.        , 1.        ,
        1.30103   ],
       [0.        , 0.        , 0.        , 1.30103   , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , 1.        , 0.        ,
        1.        ]])

In [50]:
to_tf_idf_matrix(td_matrix)

array([[0.30103   , 0.        , 0.30103   , 0.30103   , 0.        ,
        0.        ],
       [0.17609126, 0.22910001, 0.        , 0.22910001, 0.17609126,
        0.        ],
       [0.47712125, 0.        , 0.47712125, 0.        , 0.        ,
        0.        ],
       [0.        , 0.17609126, 0.17609126, 0.        , 0.26010814,
        0.22910001],
       [0.        , 0.17609126, 0.17609126, 0.        , 0.17609126,
        0.22910001],
       [0.        , 0.        , 0.        , 0.62074906, 0.47712125,
        0.        ],
       [0.        , 0.        , 0.        , 0.47712125, 0.        ,
        0.47712125]])

__d)__ We want to test our implementation on our running example. First, print the tf-idf for each word of the query ``ice beer`` with respect to each document. Second, find the two most similar documents from ``d1, d2, d3`` according to cosine similarity and print all similarity values.  __(3 pts)__

In [51]:
from numpy.linalg import norm
word2index,doc2index,index2doc,doc_word_vectors=process_docs(docs)
td_matrix = term_document_matrix(doc_word_vectors,doc2index,word2index)
tf_idf = to_tf_idf_matrix(td_matrix)
query = ("ice","beer")
for q in query:
    print("TF-IDF values for "+q+":")
    print(tf_idf[word2index[q]])
# ice = tf_idf[3]
# beer = tf_idf[1]

def cosine_sim(x, y):
    return np.dot(x, y) / (norm(x) * norm(y))

print("Cosine similarity of d1 & d2:")
print(cosine_sim(tf_idf[:,0], tf_idf[:,1]))
print("Cosine similarity of d1 & d3:")
print(cosine_sim(tf_idf[:,0], tf_idf[:,2]))
print("Cosine similarity of d2 & d3:")
print(cosine_sim(tf_idf[:,1], tf_idf[:,2]))

TF-IDF values for ice:
[0.         0.17609126 0.17609126 0.         0.26010814 0.22910001]
TF-IDF values for beer:
[0.17609126 0.22910001 0.         0.22910001 0.17609126 0.        ]
Cosine similarity of d1 & d2:
0.201730941334603
Cosine similarity of d1 & d3:
0.8732802004950573
Cosine similarity of d2 & d3:
0.2971975761024058


__e)__ In a second step we want to find the documents that are most similar to a provided query. Therefore, implement the function ``process_query`` that creates a vector represention of the query. __(5 pts)__

First, apply stemming to each query word. Second, create a vector that has an entry for each vocabulary word (words that appeared in any document), where the entry at position ``i`` indicates how often the word with id ``i`` (as indicated by ``word2index``) appears in the query. 

If ``tf`` is set to ``True``, you should transform all entries to tf-values. Similar, if ``idf`` is set to ``True``, return a vector with tf-idf values (cf. task __c)__). The computation of the idf values is based on the corresponding input term-document matrix.

In case the query contains words that are not in any of the documents, print an appropriate error message and stop the computation.

In [52]:
from nltk.stem.porter import PorterStemmer

def process_query(query: List[str], word2index: Dict[str, int], td_matrix: NDArray[NDArray[float]], tf: bool=True, idf: bool=True) -> NDArray[float]:
    """
    :param query: list of query tokens
    :param word2index: dict that maps each word to a unique id
    :param td_matrix: term-document matrix
    :param tf: computes the tf vector of the query if True
    :param idf: computes the tf-idf vector of the query if True, ignored if tf=False
    :return: vector representation of the input query (using tf(-idf))    
    """
    stemmer = PorterStemmer()
    stemmed_q = [stemmer.stem(word) for word in query]
    result = np.zeros(len(word2index))
    for q in stemmed_q:
        if q not in word2index:
            raise Exception(f"The word {q} was not found any document.")
            
        else:
            result[word2index[q]]+=1
    if tf:
        result = np.where(result > 0, 1 + np.log10(result), 0)
    if idf:
        N = td_matrix.shape[1]
        for i in range(td_matrix.shape[0]):
            result[i]*=np.log10(N/sum(np.where(td_matrix[i] > 0, 1, 0)))

    return result

In [53]:
# testing
test = ["ice", "beer"]

print(process_query(test, word2index, td_matrix,False,False))
print(process_query(test, word2index, td_matrix,True,False))
print(process_query(test, word2index, td_matrix,True,True))

[0. 1. 0. 1. 0. 0. 0.]
[0. 1. 0. 1. 0. 0. 0.]
[0.         0.17609126 0.         0.17609126 0.         0.
 0.        ]


__f)__ Implement a function ``most_similar_docs`` that gets the vector representation of a query (in terms of counts, tf values or tf-idf values) and a term-document matrix that can either contain counts, tf-values or tf-idf values.  Compute the cosine similarity between the query and all documents and return the document names and the cosine similarity values of the top-``k`` documents that are most similar to the query. The value of ``k`` should be specified by the user. __(3 pts)__

In [54]:
def most_similar_docs(query_v: NDArray[float], td_matrix: NDArray[NDArray[float]], index2doc: Dict[int, str], k: int) -> (List[str], List[float]):
    """
    :param query_v: vector representation of the input query
    :param td_matrix: term-document matrix, possibly with tf-(idf) values 
    :param index2doc: dict that maps each document id to its name
    :k: number of documents to return
    :returns:
        - list with names of the top-k most similar documents to the query, ordered by descending similarity
        - list with cosine similarities of the top-k most similar docs, ordered by descending similarity
    """
    # your code here
    if k> len(index2doc):
        k = len(index2doc)
    d = {}
    for i in range(len(index2doc)):
        sim = (cosine_sim(query_v, td_matrix[:,i]))
        d[index2doc[i]] = sim
    result = sorted(d.items(), key = lambda x:x[1], reverse=True)
    tmp1 = []
    tmp2 = []
    for i in range(k):
        tmp1.append(list(result[i])[0])
        tmp2.append(list(result[i])[1])
    return tmp1,tmp2

In [55]:
# test
td = to_tf_idf_matrix(td_matrix)
test = ['beach', 'cold', 'ice', 'cream']
q = process_query(test, word2index, td_matrix,True,True)
most_similar_docs(q,td,index2doc,3)

(['d3', 'd1', 'd2'], [1.0, 0.8732802004950573, 0.2971975761024058])

__g)__ Finally, apply your implementation to a subset of Wikipedia articles that are linked to from the English Wikipedia page of "Mannheim". __(5 + 2 + 2 + 2 pts)__

The file docs.zip contains the introduction sections of 255 Wikipedia articles, where the file names indicate
the names of the corresponding Wikipedia pages. Each file correponds to a document, where stopwords have already been removed and stemming and conversion to lower-case has been applied. Besides tokenization there is no more cleaning needed.

Read each file and create the same data structures as in task __a)__. To do this, consider the Python package ``os`` (https://docs.python.org/3/library/os.html) and specifically the function ``os.walk``. When opening and reading the files, make sure to set the encoding to ``utf-8``.

Make use of the functions that you implemented in the course of this assignment to compute and print the 5 most similar documents and their cosine similarity for the query ``mannheim university`` by considering

i. only word counts  

ii. tf values

iii. tf-idf values

In [10]:
query_wiki = ["university", "city", "ice"]


In [56]:
import os
for root, dirs, files in os.walk("docs"):
    for name in files:
        print(name)

2007 FEI European Jumping Championship
3 Liga
44th Infantry Division (United States)
7th Signal Brigade (United States)
Aachen
ABB
Adler Mannheim
Aerobus
Air pollution
Alb-Donau-Kreis
Albert Speer
Alliance 90 The Greens
Alliance for Innovation and Justice
Alsace
Alstom
Alternative for Germany
American football
American Forces Network
Arena
Art Nouveau
Augsburg
August von Kotzebue
Augustaanlage
Bad D乺kheim
Bad Rappenau
Baden-Baden
Baden-W乺ttemberg
Baden-W乺ttemberg Cooperative State University
Baroque architecture
Basel
BASF
Battalion
Benjamin Franklin Village
Bergisch Gladbach
Berlin
Bertha Benz
Bertha Benz Memorial Route
Biberach (district)
Bicycle
Bielefeld
Bild
Bochum
Bodenseekreis
Bombardier Inc
Bombing of Mannheim in World War II
Bonn
Bottrop
Braunschweig
Breisgau-Hochschwarzwald
Bremen
Bremerhaven
Buchen
Bundesgartenschau
Bundesliga
Bundesliga (baseball)
Business administration
Bydgoszcz
B乴ent Ceylan
B攂lingen (district)
Calw (district)
Carl Benz
Carl-Benz-Stadion
Carlo Grua
Caterp

In [60]:
def process_wikipedia_docs() -> (Dict[str, int], Dict[str, int], Dict[int, str], Dict[str, List[int]]):
    """
    :returns:
    - word2index: dict that maps each word to a unique id
    - doc2index: dict that maps each document to a unique id
    - index2doc: dict that maps ids to their associated document
    - doc_word_vectors: dict that maps each document name to a list of word ids that appear in it
    """
    # your code here
    docs = {}
    for root, dirs, files in os.walk("docs"):
        for name in files:
            cur = os.path.join("docs", name)
            with open(cur,encoding='utf-8') as f:
                docs[name]=f.read()
    return process_docs(docs)

word2index, doc2index, index2doc, doc_word_vectors=process_wikipedia_docs()

In [64]:
# i)
query_wiki =['mannheim', 'university']
q = process_query(query_wiki, word2index, td_matrix,False,False)
td_matrix = term_document_matrix(doc_word_vectors,doc2index,word2index)
most_similar_docs(q,td_matrix,index2doc,5)

(['Heidelberg University Faculty of Medicine in Mannheim',
  'Mannheim (disambiguation)',
  'Mannheim Palace',
  'Augustaanlage',
  'Kunsthalle Mannheim'],
 [0.40931560807732065,
  0.39223227027636803,
  0.34503277967117707,
  0.3086066999241838,
  0.29138575870717925])

In [65]:
# ii)
q = process_query(query_wiki, word2index, td_matrix,True,False)
td = to_tf_idf_matrix(td_matrix, False)
most_similar_docs(q,td,index2doc,5)

(['Mannheim (disambiguation)',
  'Mannheim Palace',
  'Heidelberg University Faculty of Medicine in Mannheim',
  'Mannheim Rangierbahnhof',
  'Augustaanlage'],
 [0.281338421216347,
  0.2740006740462574,
  0.2673707615492943,
  0.22421615584111737,
  0.212782808600512])

In [66]:
# iii)
q = process_query(query_wiki, word2index, td_matrix,True,True)
td = to_tf_idf_matrix(td_matrix, True)
most_similar_docs(q,td,index2doc,5)

(['Mannheim (disambiguation)',
  'Mannheim Palace',
  'Heidelberg University Faculty of Medicine in Mannheim',
  'Mannheim Rangierbahnhof',
  'Bombing of Mannheim in World War II'],
 [0.17837437258082361,
  0.1431133714249375,
  0.12457000565595781,
  0.11757716481970117,
  0.10974076404633058])